**AutoML OSS入門（3）**
# 遺伝的プログラミングによるAutoML OSS「TPOT」

本ノートブックの紹介記事と併せてご覧ください。
- ＠IT連載 AutoML OSS入門（3）- 第3回「遺伝的プログラミングによるAutoML OSS「TPOT」」

また、ノートブックの扱い方やタイタニックの生存予測データについては、連載記事の第1回を参照してください。
- [＠IT連載 AutoML OSS入門（１）- 第1回「機械学習モデル構築作業の煩雑さを解消する「AutoML」とは――歴史、動向、利用のメリットを整理する」](https://www.atmarkit.co.jp/ait/articles/2107/02/news006.html)

## タイタニックデータでAutoML

### セットアップ

In [1]:
# tpotのインストール
!pip install tpot

     |████████████████████████████████| 87 kB 2.5 MB/s 
     |████████████████████████████████| 166.7 MB 17 kB/s 
     |████████████████████████████████| 160 kB 50.1 MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11952 sha256=60af6c5c4343c6b89836e8e7054d2c4ca06482f341c23d45f58943e4da78afc0
  Stored in directory: /root/.cache/pip/wheels/e2/d2/79/eaf81edb391e27c87f51b8ef901ecc85a5363dc96b8b8d71e3
Successfully built stopit
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [2]:
!tpot --version

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
TPOT 0.11.7


### データのロード
第1回「AutoMLとは」の「タイタニックの生存予測データの取得方法と解説」の章を参照してください。

In [3]:
# データの準備
!wget -N https://github.com/aiq2020-tw/automl-notebooks/raw/main/titanic.zip
!unzip titanic.zip

--2021-09-15 09:06:41--  https://github.com/aiq2020-tw/automl-notebooks/raw/main/titanic.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aiq2020-tw/automl-notebooks/main/titanic.zip [following]
--2021-09-15 09:06:41--  https://raw.githubusercontent.com/aiq2020-tw/automl-notebooks/main/titanic.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34877 (34K) [application/zip]
Saving to: ‘titanic.zip’

titanic.zip         100%[===================>]  34.06K  --.-KB/s    in 0.002s  

Last-modified header missing -- time-stamps turned off.
2021-09-15 09:06:41 (14.5 MB/s) - ‘titanic.zip’ saved [3487

In [4]:
# 学習データ/テストデータの読み込み
import pandas as pd
train_df = pd.read_csv('train.csv')
print('Train data shape:', str(train_df.shape))
test_df = pd.read_csv('test.csv')
print('Test data shape:', str(test_df.shape))

Train data shape: (891, 12)
Test data shape: (418, 11)


### データの前処理

In [5]:
# 「PassengerId」列と「Name」列は今回は使用しない
X_train = train_df.drop(['PassengerId', 'Name'], axis=1)
X_test = test_df.drop(['PassengerId', 'Name'], axis=1)
# 目的変数
y_train = X_train.pop('Survived')

In [6]:
# TPOTでは説明変数を数値型にする必要があるため変換
from sklearn.preprocessing import LabelEncoder

list_cols = ['Sex', 'Ticket', 'Cabin', 'Embarked']
for col in list_cols:
    X_train[col] = X_train[col].fillna('missing')
    X_test[col] = X_test[col].fillna('missing')
    target_column = pd.concat([X_train[col], X_test[col]])
    le = LabelEncoder()
    le.fit(target_column)
    X_train[col] = le.transform(X_train[col])
    X_train[col] = X_train[col].astype('category')
    X_test[col] = le.transform(X_test[col])
    X_test[col] = X_test[col].astype('category')
X_train.dtypes

Pclass         int64
Sex         category
Age          float64
SibSp          int64
Parch          int64
Ticket      category
Fare         float64
Cabin       category
Embarked    category
dtype: object

### モデルの学習と評価

In [7]:
# モデルの学習
from tpot import TPOTClassifier
tpot = TPOTClassifier(verbosity=2, generations=10, population_size=5, random_state=42)
tpot.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


Imputing missing values in feature set


Optimization Progress:   0%|          | 0/55 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8227167158370472

Generation 2 - Current best internal CV score: 0.8227167158370472

Generation 3 - Current best internal CV score: 0.8272048207896555

Generation 4 - Current best internal CV score: 0.8316678174628084

Generation 5 - Current best internal CV score: 0.8372983491306257

Generation 6 - Current best internal CV score: 0.8372983491306257

Generation 7 - Current best internal CV score: 0.837323457410081

Generation 8 - Current best internal CV score: 0.837323457410081

Generation 9 - Current best internal CV score: 0.837323457410081

Generation 10 - Current best internal CV score: 0.837323457410081

Best pipeline: XGBClassifier(input_matrix, learning_rate=0.01, max_depth=9, min_child_weight=1, n_estimators=100, n_jobs=1, subsample=0.8, verbosity=0)


TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=10,
               log_file=None, max_eval_time_mins=5, max_time_mins=None,
               memory=None, mutation_rate=0.9, n_jobs=1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=5,
               random_state=42, scoring=None, subsample=1.0, template=None,
               use_dask=False, verbosity=2, warm_start=False)

In [8]:
# 最も精度の良かったパイプラインを表示
tpot.fitted_pipeline_

Pipeline(memory=None,
         steps=[('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints='', learning_rate=0.01,
                               max_delta_step=0, max_depth=9,
                               min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=1, num_parallel_tree=1,
                               objective='binary:logistic', random_state=42,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               subsample=0.8, tree_method='exact',
                               use_label_encoder=True, validate_parameters=1,
                               verbosi

In [9]:
# 最適化の過程で生成された他のパイプライン
tpot.evaluated_individuals_

{'BernoulliNB(input_matrix, BernoulliNB__alpha=0.1, BernoulliNB__fit_prior=True)': {'crossover_count': 0,
  'generation': 5,
  'internal_cv_score': 0.785606678802335,
  'mutation_count': 2,
  'operator_count': 1,
  'predecessor': ('ExtraTreesClassifier(input_matrix, ExtraTreesClassifier__bootstrap=True, ExtraTreesClassifier__criterion=entropy, ExtraTreesClassifier__max_features=0.6000000000000001, ExtraTreesClassifier__min_samples_leaf=15, ExtraTreesClassifier__min_samples_split=10, ExtraTreesClassifier__n_estimators=100)',)},
 'DecisionTreeClassifier(input_matrix, DecisionTreeClassifier__criterion=gini, DecisionTreeClassifier__max_depth=5, DecisionTreeClassifier__min_samples_leaf=9, DecisionTreeClassifier__min_samples_split=10)': {'crossover_count': 0,
  'generation': 1,
  'internal_cv_score': 0.8014123407193521,
  'mutation_count': 1,
  'operator_count': 1,
  'predecessor': ('GaussianNB(input_matrix)',)},
 'ExtraTreesClassifier(ExtraTreesClassifier(input_matrix, ExtraTreesClassifier_

### テストデータの予測

In [10]:
# 予測
predictions = tpot.predict(X_test)
predictions

Imputing missing values in feature set


array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [11]:
# 投稿用のファイル作成
output = pd.DataFrame({'PassengerID': test_df.PassengerId,
                       'Survived': predictions})
output.to_csv('submission.csv', index=False)

## その他の機能と応用

### パイプラインで利用するオペレーターの制御
オペレーターおよびパラメーターの探索範囲を指定してみましょう。

In [12]:
# オペレータおよびパラメータを指定した辞書を作成
tpot_config = {
    'sklearn.naive_bayes.GaussianNB': {
    },
 
    'sklearn.naive_bayes.BernoulliNB': {
        'alpha': [1e-3, 1e-2, 1e-1, 1., 10., 100.],
        'fit_prior': [True, False]
    },
 
    'sklearn.naive_bayes.MultinomialNB': {
        'alpha': [1e-3, 1e-2, 1e-1, 1., 10., 100.],
        'fit_prior': [True, False]
    }
}

In [13]:
# 作成した辞書を使ったモデルの学習
tpot = TPOTClassifier(config_dict=tpot_config, verbosity=2, generations=10, random_state=42)
tpot.fit(X_train, y_train)

Imputing missing values in feature set


Optimization Progress:   0%|          | 0/1100 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7867679367271359

Generation 2 - Current best internal CV score: 0.7957629778419435

Generation 3 - Current best internal CV score: 0.7957629778419435

Generation 4 - Current best internal CV score: 0.7957629778419435

Generation 5 - Current best internal CV score: 0.7957629778419435

Generation 6 - Current best internal CV score: 0.7957629778419435

Generation 7 - Current best internal CV score: 0.7957629778419435

Generation 8 - Current best internal CV score: 0.7957629778419435

Generation 9 - Current best internal CV score: 0.7957629778419435

Generation 10 - Current best internal CV score: 0.7957629778419435

Best pipeline: BernoulliNB(GaussianNB(input_matrix), alpha=100.0, fit_prior=True)


TPOTClassifier(config_dict={'sklearn.naive_bayes.BernoulliNB': {'alpha': [0.001,
                                                                          0.01,
                                                                          0.1,
                                                                          1.0,
                                                                          10.0,
                                                                          100.0],
                                                                'fit_prior': [True,
                                                                              False]},
                            'sklearn.naive_bayes.GaussianNB': {},
                            'sklearn.naive_bayes.MultinomialNB': {'alpha': [0.001,
                                                                            0.01,
                                                                            0.1,
                                    

In [14]:
# 最も精度の良かったパイプラインを表示
tpot.fitted_pipeline_

Pipeline(memory=None,
         steps=[('stackingestimator',
                 StackingEstimator(estimator=GaussianNB(priors=None,
                                                        var_smoothing=1e-09))),
                ('bernoullinb',
                 BernoulliNB(alpha=100.0, binarize=0.0, class_prior=None,
                             fit_prior=True))],
         verbose=False)

指定したオペレーターおよびパラメーターだけが探索範囲となり、比較的早期に学習が完了しました。

### 最適化したパイプラインの出力

以下のコードを実行すると、最適化された学習済みパイプラインをpythonスクリプトとして出力することができます。

In [15]:
tpot.export('tpot_titanic_pipeline.py')

以上で、タイタニックの生存予測データを使ったTPOTの紹介は終わりです。